In [1]:
import tensorflow as tf
from tensorflow import keras                    
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import shutil
import numpy as np
import os
from PIL import Image
import pandas



In [36]:
train_data_dir = "/Users/tony/Documents/research_projects/aiGenerated/dataset/train"
test_data_dir = "/Users/tony/Documents/research_projects/aiGenerated/dataset/archive-2/real_vs_fake/real-vs-fake/testF"
validation_data_dir ="/Users/tony/Documents/research_projects/aiGenerated/dataset/validation"
IMG_WIDTH,IMG_HEIGHT = 256,256
input_shape = (IMG_WIDTH,IMG_HEIGHT,3)

# *Data augmentation*

In [37]:
#data generator for RGB images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)
batch_size = 32

#Define data generators for RGB images with augmentations
datagen_augmented = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    brightness_range = [0.8, 1.2],
    preprocessing_function = lambda img: img + np.random.normal(loc=0.0, scale=0.05, size=img.shape),

    fill_mode = "reflect"
    
)
train_generator = datagen_augmented.flow_from_directory(
    train_data_dir, 
    target_size=(IMG_WIDTH,IMG_HEIGHT),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42, 
    subset='training',
)

test_generator = datagen_augmented.flow_from_directory(
    test_data_dir,
    target_size = (IMG_WIDTH,IMG_HEIGHT),
    batch_size = 4,
    class_mode ='categorical',
    shuffle = False,
    
)
validation_generator = datagen_augmented.flow_from_directory(
    validation_data_dir,
    target_size = (IMG_WIDTH,IMG_HEIGHT),
    batch_size = 16,
    shuffle = True,
    seed = 42,
    class_mode = 'categorical'
    
)

Found 90000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 10000 images belonging to 2 classes.


In [4]:
class_indices = train_generator.class_indices
print(class_indices)

{'FAKE': 0, 'REAL': 1}


# *No of images for each class in the training dataset*

In [5]:
#No of images for each class in the training dataset
classes = [class_name for class_name in os.listdir(train_data_dir) if os.path.isdir(os.path.join(train_data_dir,class_name))]
for class_name in classes:
    class_path = os.path.join(train_data_dir,class_name)
    num_images = len(os.listdir(class_path))
    print(f"class: {class_name},Number of images: {num_images}")

class: REAL,Number of images: 45000
class: FAKE,Number of images: 45000


# *Shape of the images in Train Generator*

In [6]:
#get a batch of images and labels from the train_generator
batch = train_generator.__next__()

#Iterate throug the batch to check image shapes
for i in range(len(batch[0])):
    img = batch[0][i]
    label = batch[1][i]

    #Get image shape and channels
    height,width,channels = img.shape

    #Display image shape and channels
    print(f"Image {i+1} - Shape: {width}x{height}x{channels},label: {label}")
    
    

Image 1 - Shape: 32x32x3,label: [0. 1.]
Image 2 - Shape: 32x32x3,label: [1. 0.]
Image 3 - Shape: 32x32x3,label: [0. 1.]
Image 4 - Shape: 32x32x3,label: [0. 1.]
Image 5 - Shape: 32x32x3,label: [0. 1.]
Image 6 - Shape: 32x32x3,label: [1. 0.]
Image 7 - Shape: 32x32x3,label: [0. 1.]
Image 8 - Shape: 32x32x3,label: [1. 0.]
Image 9 - Shape: 32x32x3,label: [0. 1.]
Image 10 - Shape: 32x32x3,label: [1. 0.]
Image 11 - Shape: 32x32x3,label: [1. 0.]
Image 12 - Shape: 32x32x3,label: [0. 1.]
Image 13 - Shape: 32x32x3,label: [0. 1.]
Image 14 - Shape: 32x32x3,label: [1. 0.]
Image 15 - Shape: 32x32x3,label: [0. 1.]
Image 16 - Shape: 32x32x3,label: [0. 1.]
Image 17 - Shape: 32x32x3,label: [1. 0.]
Image 18 - Shape: 32x32x3,label: [1. 0.]
Image 19 - Shape: 32x32x3,label: [0. 1.]
Image 20 - Shape: 32x32x3,label: [0. 1.]
Image 21 - Shape: 32x32x3,label: [0. 1.]
Image 22 - Shape: 32x32x3,label: [1. 0.]
Image 23 - Shape: 32x32x3,label: [1. 0.]
Image 24 - Shape: 32x32x3,label: [1. 0.]
Image 25 - Shape: 32x32x3

# *Number of images in test generator*

In [38]:
classes  = [class_name for class_name in os.listdir(test_data_dir) if os.path.isdir(os.path.join(test_data_dir,class_name))]
for class_name in classes:
    class_path = os.path.join(test_data_dir,class_name)
    num_images = len(os.listdir(class_path))
    print(f"Class: {class_name},Number of images: {num_images}")
    

Class: real,Number of images: 10000
Class: fake,Number of images: 10000


# *Shape of the images in test generator*

In [39]:
#get a batch of images and labels from the train_generator 
batch = test_generator.__next__()

for i in range(len(batch[0])):
    img = batch[0][i] #Image data
    label = batch[1][i] #image label

    height,width,channels = img.shape

    print(f"Image {i+1} - Shape: {width}x{height}x{channels},label: {label}")

Image 1 - Shape: 256x256x3,label: [1. 0.]
Image 2 - Shape: 256x256x3,label: [1. 0.]
Image 3 - Shape: 256x256x3,label: [1. 0.]
Image 4 - Shape: 256x256x3,label: [1. 0.]


In [31]:
classes  = [class_name for class_name in os.listdir(validation_data_dir) if os.path.isdir(os.path.join(test_data_dir,class_name))]
for class_name in classes:
    class_path = os.path.join(test_data_dir,class_name)
    num_images = len(os.listdir(class_path))
    print(f"Class: {class_name},Number of images: {num_images}")

Class: REAL,Number of images: 10000
Class: FAKE,Number of images: 10000


In [10]:
#get a batch of images and labels from the train_generator 
batch = validation_generator.__next__()

for i in range(len(batch[0])):
    img = batch[0][i] #Image data
    label = batch[1][i] #image label

    height,width,channels = img.shape

    print(f"Image {i+1} - Shape: {width}x{height}x{channels},label: {label}")

Image 1 - Shape: 32x32x3,label: [0. 1.]
Image 2 - Shape: 32x32x3,label: [1. 0.]
Image 3 - Shape: 32x32x3,label: [1. 0.]
Image 4 - Shape: 32x32x3,label: [1. 0.]
Image 5 - Shape: 32x32x3,label: [1. 0.]
Image 6 - Shape: 32x32x3,label: [0. 1.]
Image 7 - Shape: 32x32x3,label: [1. 0.]
Image 8 - Shape: 32x32x3,label: [0. 1.]
Image 9 - Shape: 32x32x3,label: [0. 1.]
Image 10 - Shape: 32x32x3,label: [1. 0.]
Image 11 - Shape: 32x32x3,label: [1. 0.]
Image 12 - Shape: 32x32x3,label: [0. 1.]
Image 13 - Shape: 32x32x3,label: [0. 1.]
Image 14 - Shape: 32x32x3,label: [0. 1.]
Image 15 - Shape: 32x32x3,label: [1. 0.]
Image 16 - Shape: 32x32x3,label: [0. 1.]


# *check for GPU availability*

In [11]:
#Check for GPU availability
if tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices ('GPU')[0],True)
    print("GPU device configured")
else:
    print("No GPU device found")

GPU device configured


# *Model Checkpoint*

In [12]:
model_dir = "/Users/tony/Documents/research_projects/aiGenerated/model/checkpoints6"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

checkpoint_filename = "cp.weights.h5"
checkpoint_path = os.path.join(model_dir,checkpoint_filename)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 save_weights_only = True,
                                                 save_best_only = True,
                                                 monitor = "val_accuracy",
                                                 mode = "max",
                                                 verbose = 1)

In [13]:
checkpoint_path

'/Users/tony/Documents/research_projects/aiGenerated/model/checkpoints6/cp.weights.h5'

# *MODEL DESIGN*

In [40]:
def hybrid_cnn_transformer(input_shape = (256,256,3)):
    inputs  = keras.Input(shape = input_shape)

    #CNN Features Extraction
    x = layers.Conv2D(32,(3,3),activation = "relu",padding = "same",kernel_regularizer = regularizers.l2(1e-4))(inputs)
    #x = layers.LeakyReLU(negative_slope=0.1)(x)#new relu
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64,(3,3),activation = "relu",padding = "same",kernel_regularizer = regularizers.l2(1e-4))(x)
    x = layers.LeakyReLU(negative_slope=0.1)(x)#new
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Dropout(0.4)(x) #new dropout

    x = layers.Conv2D(128,(3,3),activation = "relu",padding = "same",kernel_regularizer = regularizers.l2(1e-4))(x)
    #x = layers.LeakyReLU(negative_slope=0.1)(x)#new
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256,(3,3),activation = "relu",padding = "same",kernel_regularizer = regularizers.l2(1e-4))(x)
    x = layers.LeakyReLU(negative_slope=0.1)(x)#new relu
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Dropout(0.4)(x)#new dropi

    patches = layers.Reshape((-1,256))(x)
    patches = layers.Dense(256,kernel_initializer = "glorot_uniform")(patches)

    #Transformer block 1
    norm_patches = layers.LayerNormalization()(patches)
    transformer_layer = layers.MultiHeadAttention(num_heads = 8,key_dim = 128,dropout=0.3)(norm_patches,norm_patches)
    transformer_layer = layers.Add()([transformer_layer,patches]) #residual connection
    transformer_layer = layers.LayerNormalization()(transformer_layer)
    transformer_layer = layers.Dense(256,activation = "relu",kernel_regularizer = regularizers.l2(1e-4))(transformer_layer)
    transformer_layer = layers.Dropout(0.4)(transformer_layer)

    #Transformer block 2
    """norm_transformer = layers.LayerNormalization()(transformer_layer)
    transformer_layer = layers.MultiHeadAttention(num_heads=8,key_dim = 128,dropout = 0.3)(norm_transformer,norm_transformer)
    transformer_layer = layers.Add()([transformer_layer,norm_transformer])
    transformer_layer = layers.Dense(256,activation = "relu",kernel_regularizer=regularizers.l2(1e-4))(transformer_layer)
    transformer_layer = layers.Dropout(0.4)(transformer_layer)"""

    #classification Head
    features = layers.GlobalAveragePooling1D()(transformer_layer)
    features = layers.Dropout(0.4)(features)
    outputs = layers.Dense(2,activation = "softmax")(features)

    model = keras.Model(inputs,outputs)
    # model.compile(
    #     optimizer=Adam(learning_rate=0.0001,clipnorm = 1.0),
    #     loss = "categorical_crossentropy",
    #     metrics = ["accuracy"]
        
    # )
    return model

model = hybrid_cnn_transformer()
model.summary()
    
    
    
    


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 256, 256,  │        896 │ input_layer_6[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        128 │ conv2d_24[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_25 (Conv2D)  │ (None, 256, 256,  │     18,496 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_12      │ (None, 256, 256,  │          0 │ conv2d_25[0][0]   │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ leaky_re_lu_12[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_30          │ (None, 128, 128,  │          0 │ max_pooling2d_12… │
│ (Dropout)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_26 (Conv2D)  │ (None, 128, 128,  │     73,856 │ dropout_30[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_26[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_27 (Conv2D)  │ (None, 128, 128,  │    295,168 │ batch_normalizat… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_13      │ (None, 128, 128,  │          0 │ conv2d_27[0][0]   │
│ (LeakyReLU)         │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │      1,024 │ leaky_re_lu_13[0… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_13    │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_31          │ (None, 64, 64,    │          0 │ max_pooling2d_13… │
│ (Dropout)           │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_6 (Reshape) │ (None, 4096, 256) │          0 │ dropout_31[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 4096, 256) │     65,792 │ reshape_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,575,362 (6.01 MB)

 Trainable params: 1,574,402 (6.01 MB)

 Non-trainable params: 960 (3.75 KB)

# *Training start here*

In [20]:
"""lr_schedule = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6
)"""

callbacks = [
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
    #ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-4)
]


In [16]:
history = model.fit(
    train_generator,
    epochs = 20,
    validation_data = validation_generator,
    callbacks = [callbacks,cp_callback]
)

Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2025-02-12 09:01:33.892463: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.7575 - loss: 0.5641  
Epoch 1: val_accuracy improved from -inf to 0.85530, saving model to /Users/tony/Documents/research_projects/aiGenerated/model/checkpoints5/cp.weights.h5
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 163s 56ms/step - accuracy: 0.7575 - loss: 0.5640 - val_accuracy: 0.8553 - val_loss: 0.4062
Epoch 2/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.8073 - loss: 0.5742  
Epoch 2: val_accuracy improved from 0.85530 to 0.87130, saving model to /Users/tony/Documents/research_projects/aiGenerated/model/checkpoints5/cp.weights.h5
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 159s 56ms/step - accuracy: 0.8072 - loss: 0.5742 - val_accuracy: 0.8713 - val_loss: 0.4162
Epoch 3/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.8217 - loss: 0.5624  
Epoch 3: val_accuracy did not improve from 0.87130
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 164s 58ms/step - accuracy: 0.8217 - loss: 0.5624 - val_accuracy: 0.8588 - val_loss: 0.4176
Epoch 4/20

# *saved history model*

In [20]:
# Save the training history
initial_epoch = 0  # or the actual initial epoch of the first training session
saved_history = {
    'loss': history.history['loss'],
    'accuracy': history.history['accuracy'],
    'val_loss': history.history['val_loss'],
    'val_accuracy': history.history['val_accuracy'],
    # Add other metrics as needed
}
np.save("/Users/tony/Documents/research_projects/aiGenerated/model/model_history.npy", saved_history)

NameError: name 'history' is not defined

In [23]:
import pandas as pd
pd.DataFrame(history.history).to_csv("/Users/tony/Documents/research_projects/aiGenerated/model/epoch.csv")

# *Testing*

In [41]:

model = hybrid_cnn_transformer()

checkpoint_path = "/Users/tony/Documents/research_projects/aiGenerated/model/checkpoints5/cp.weights.h5"

try:
    model.load_weights(checkpoint_path)
    print("weights loaded successfully")
except Exception as e:
    print(f"Failed to load weights. Error{e}")

weights loaded successfully


In [42]:
#Evaluate the model on the test
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions,axis =1)
true_classes = test_generator.classes

print("Predidcted classes:",predicted_classes[-10:])
print("True classes:",true_classes[-10:])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5000/5000 ━━━━━━━━━━━━━━━━━━━━ 1099s 219ms/step
Predidcted classes: [0 0 0 0 0 1 0 0 0 1]
True classes: [1 1 1 1 1 1 1 1 1 1]


In [43]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, log_loss, jaccard_score
print(f"Accuracy: {accuracy_score(true_classes,predicted_classes)}")
print(f"Precision: {precision_score(true_classes,predicted_classes,average = "weighted")}")
print(f"Recall: {f1_score(true_classes,predicted_classes,average = "weighted")}")
print(f"Jaccard Score {jaccard_score(true_classes,predicted_classes,average="weighted")}")

Accuracy: 0.4854
Precision: 0.4794089999368731
Recall: 0.4450327416123456
Jaccard Score 0.2982290482375963


# *Retraining the model*

In [21]:
from tensorflow.keras.models import load_model
model = hybrid_cnn_transformer()
model.load_weights("/Users/tony/Documents/research_projects/aiGenerated/model/checkpoints5/cp.weights.h5")

def gradual_unfreezing(model,train_generator,validation_generator):
    """Gradually  unfreeze and retrain layers"""
    for layer in model.layers:
        layer.trainable = False

    for i in range(len(model.layers)-1,-1,-1):
        print(f"Unfreezing layer: {model.layers[i].name}")
        model.layers[i].trainable = True

    model.compile(
        optimizer=Adam(learning_rate=0.00001,clipnorm = 1.0),
        loss = "categorical_crossentropy",
        metrics = ["accuracy"]
    )

    history = model.fit(
    train_generator,
    epochs = 20,
    batch_size = 16,
    validation_data = validation_generator,
    callbacks = [callbacks,cp_callback]
)
    return model
model = gradual_unfreezing(model,train_generator,validation_generator)


Unfreezing layer: dense_11
Unfreezing layer: dropout_19
Unfreezing layer: global_average_pooling1d_3
Unfreezing layer: dropout_18
Unfreezing layer: dense_10
Unfreezing layer: layer_normalization_7
Unfreezing layer: add_3
Unfreezing layer: multi_head_attention_3
Unfreezing layer: layer_normalization_6
Unfreezing layer: dense_9
Unfreezing layer: reshape_3
Unfreezing layer: dropout_16
Unfreezing layer: max_pooling2d_7
Unfreezing layer: batch_normalization_15
Unfreezing layer: leaky_re_lu_7
Unfreezing layer: conv2d_15
Unfreezing layer: batch_normalization_14
Unfreezing layer: conv2d_14
Unfreezing layer: dropout_15
Unfreezing layer: max_pooling2d_6
Unfreezing layer: batch_normalization_13
Unfreezing layer: leaky_re_lu_6
Unfreezing layer: conv2d_13
Unfreezing layer: batch_normalization_12
Unfreezing layer: conv2d_12
Unfreezing layer: input_layer_3
Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9106 - loss: 0.3053  
Epoch 1: val_accuracy improved from -inf to 0.92470, saving model to /Users/tony/Documents/research_projects/aiGenerated/model/checkpoints6/cp.weights.h5
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 166s 57ms/step - accuracy: 0.9106 - loss: 0.3053 - val_accuracy: 0.9247 - val_loss: 0.2738
Epoch 2/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9138 - loss: 0.2981  
Epoch 2: val_accuracy improved from 0.92470 to 0.92570, saving model to /Users/tony/Documents/research_projects/aiGenerated/model/checkpoints6/cp.weights.h5
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 158s 56ms/step - accuracy: 0.9138 - loss: 0.2981 - val_accuracy: 0.9257 - val_loss: 0.2672
Epoch 3/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9131 - loss: 0.2969  
Epoch 3: val_accuracy did not improve from 0.92570
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 163s 58ms/step - accuracy: 0.9131 - loss: 0.2969 - val_accuracy: 0.9255 - val_loss: 0.2661
Epoch 4/20

In [22]:
#Evaluate the model on the test
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions,axis =1)
true_classes = test_generator.classes

print("Predidcted classes:",predicted_classes[-10:])
print("True classes:",true_classes[-10:])

5000/5000 ━━━━━━━━━━━━━━━━━━━━ 23s 5ms/step
Predidcted classes: [1 1 1 1 1 1 1 1 1 0]
True classes: [1 1 1 1 1 1 1 1 1 1]


# *Metrics Evaluation*

In [23]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, log_loss, jaccard_score
print(f"Accuracy: {accuracy_score(true_classes,predicted_classes)}")
print(f"Precision: {precision_score(true_classes,predicted_classes,average = "weighted")}")
print(f"Recall: {f1_score(true_classes,predicted_classes,average = "weighted")}")
print(f"Jaccard Score {jaccard_score(true_classes,predicted_classes,average="weighted")}")

Accuracy: 0.9314
Precision: 0.9319230761221068
Recall: 0.9313792243739714
Jaccard Score 0.8715736606204327
